Source code from https://github.com/google-research/bert

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_data = pd.read_csv("/content/gdrive/My Drive/NDSC2019/data/train.csv")

In [0]:
import re
def clean_text(sentence):
  cleaned = sentence
  ## Remove numbers/words contain number
  cleaner = lambda x : re.sub('^\d|\.|-|_', '', x)
  #cleaned = re.sub('[^a-zA-Z ]', '', cleaned)
  words = cleaned.split()
  cleaned = ' '.join([cleaner(word) for word in words if len(word) > 1])
  return cleaned

In [0]:
!pip install bert-tensorflow

    100% |████████████████████████████████| 71kB 3.4MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

W0322 05:59:38.892554 140079783819136 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = '/content/gdrive/My Drive/NDSC2019/code'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /content/gdrive/My Drive/NDSC2019/code *****


In [0]:
text = train_data['title']
text = text.apply(clean_text)
y = train_data['Category']

In [0]:
from sklearn.model_selection import train_test_split

## Set test_size 0.20 for validation
text_train, text_val, y_train, y_val = train_test_split(text, y, test_size=0.0, random_state = 101)

In [0]:
train = pd.DataFrame({'text':text_train, 'label':y_train})
val = pd.DataFrame({'text':text_val, 'label':y_val})

In [0]:
train.head()

,label,text
664545,35,sale baru terlaris asus zenfone max pro m1 2 z...
467249,27,bayar di tempat sweater kaos hoodie wanita len...
606650,39,brandcode b230 dual sim card putih
92020,3,bb powder ponds magic biru
146728,9,terbaru hourglass veil mineral primer spf 5 0ml


In [0]:
train.shape

(666615, 2)

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_Input = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['text'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)
test_Input = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['text'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0322 06:01:18.369006 140079783819136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0322 06:01:21.007259 140079783819136 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
label_list = list(range(0,58))

In [0]:
# We'll set sequences to be at most 20 tokens long.
MAX_SEQ_LENGTH = 20
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_Input, label_list, MAX_SEQ_LENGTH, tokenizer)
#test_features = bert.run_classifier.convert_examples_to_features(test_Input, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 666615


I0322 06:04:49.931653 140079783819136 run_classifier.py:774] Writing example 0 of 666615


INFO:tensorflow:*** Example ***


I0322 06:04:49.942613 140079783819136 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0322 06:07:39.180020 140079783819136 run_classifier.py:774] Writing example 640000 of 666615


INFO:tensorflow:Writing example 650000 of 666615


I0322 06:07:41.879018 140079783819136 run_classifier.py:774] Writing example 650000 of 666615


INFO:tensorflow:Writing example 660000 of 666615


I0322 06:07:44.524034 140079783819136 run_classifier.py:774] Writing example 660000 of 666615


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)

        return {"eval_accuracy": accuracy}
            

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
num_train_steps

41663

In [0]:
len(train_features)

666615

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/My Drive/NDSC2019/code', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f66593ed3c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0322 06:10:02.378391 140079783819136 estimator.py:201] Using config: {'_model_dir': '/content/gdrive/My Drive/NDSC2019/code', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f66593ed3c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
from datetime import datetime

In [0]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_train_steps)
eval_spec = tf.estimator.EvalSpec(input_fn=test_input_fn)

In [0]:
#train on full data
print(f'Beginning Training!')
current_time = datetime.now()
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Not using Distribute Coordinator.


I0322 01:24:12.038777 140461667964800 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


I0322 01:24:12.044305 140461667964800 training.py:610] Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 500 or save_checkpoints_secs None.


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 33330: /content/gdrive/My Drive/NDSC2019/code/model.ckpt-33330


I0322 05:55:03.282294 140461667964800 estimator.py:2039] Saving 'checkpoint_path' summary for global step 33330: /content/gdrive/My Drive/NDSC2019/code/model.ckpt-33330


INFO:tensorflow:Loss for final step: 0.70087993.


I0322 05:55:03.890287 140461667964800 estimator.py:359] Loss for final step: 0.70087993.


Training took time  4:30:51.855998


In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0322 06:11:27.525887 140079783819136 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0322 06:11:31.067510 140079783819136 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 41663 into /content/gdrive/My Drive/NDSC2019/code/model.ckpt.


I0322 10:45:43.401243 140079783819136 basic_session_run_hooks.py:594] Saving checkpoints for 41663 into /content/gdrive/My Drive/NDSC2019/code/model.ckpt.


INFO:tensorflow:Loss for final step: 0.5983216.


I0322 10:45:55.947948 140079783819136 estimator.py:359] Loss for final step: 0.5983216.


Training took time  4:35:36.113907


In [0]:
test_data = pd.read_csv("/content/gdrive/My Drive/NDSC2019/data/test.csv")

In [0]:
text_test = test_data['title']
text_test = text_test.apply(clean_text)
#y = train_data['Category']

In [0]:
test = pd.DataFrame({'text':text_test, 'label': labels})

In [0]:
labels = np.ones(172402) #dummy labels

In [0]:
test_Input = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['text'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

In [0]:
len(test_Input)

172402

In [0]:
label_list = list(range(0,58))

In [0]:
test_features = bert.run_classifier.convert_examples_to_features(test_Input, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 172402


I0322 10:53:42.568879 140079783819136 run_classifier.py:774] Writing example 0 of 172402


INFO:tensorflow:*** Example ***


I0322 10:53:42.577850 140079783819136 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0322 10:54:22.646748 140079783819136 run_classifier.py:774] Writing example 150000 of 172402


INFO:tensorflow:Writing example 160000 of 172402


I0322 10:54:24.794807 140079783819136 run_classifier.py:774] Writing example 160000 of 172402


INFO:tensorflow:Writing example 170000 of 172402


I0322 10:54:26.936380 140079783819136 run_classifier.py:774] Writing example 170000 of 172402


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
predictions = list(estimator.predict(input_fn=test_input_fn))

INFO:tensorflow:Calling model_fn.


I0322 10:54:53.564514 140079783819136 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0322 10:54:56.705988 140079783819136 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

I0322 10:54:57.662704 140079783819136 saver.py:1270] Restoring parameters from /content/gdrive/My Drive/NDSC2019/code/model.ckpt-41663


INFO:tensorflow:Running local_init_op.


I0322 10:54:59.045971 140079783819136 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0322 10:54:59.151679 140079783819136 session_manager.py:493] Done running local_init_op.


In [0]:
predictions

In [0]:
np.save('bert.npy', np.array(predictions))

In [0]:
np.load('bert.npy')

array([{'probabilities': array([ -6.787135  ,  -2.840641  ,  -7.1898174 ,  -5.7507195 ,
        -3.8871999 ,  -0.13551372,  -8.415564  ,  -5.0939517 ,
        -6.3905787 ,  -3.5187478 ,  -8.342289  ,  -7.7674975 ,
        -6.7988677 ,  -7.381596  ,  -8.024819  ,  -9.485347  ,
        -7.9830985 ,  -9.118607  ,  -9.556837  ,  -9.664669  ,
       -10.059684  ,  -9.752948  ,  -9.9773035 ,  -9.039009  ,
       -11.070967  , -10.454379  ,  -8.389034  ,  -9.378942  ,
       -10.498659  ,  -9.536549  , -10.558292  , -10.121701  ,
       -10.786078  ,  -9.904985  ,  -9.734886  , -10.544292  ,
        -9.427234  , -10.5938015 ,  -9.42785   ,  -9.960381  ,
       -10.034269  , -10.690953  , -10.433159  ,  -9.340779  ,
       -10.179315  , -11.232719  , -10.854505  , -10.317446  ,
       -10.032926  , -11.238074  , -10.594535  ,  -9.550049  ,
       -11.2015085 , -11.586504  , -10.270436  , -11.702939  ,
       -11.203889  , -10.595177  ], dtype=float32), 'labels': 5},
       {'probabilities': ar

In [0]:
labels = [x['labels'] for x in predictions]

In [0]:
labels

In [0]:
test_data['Category'] = np.array(labels)

In [0]:
submission = test_data[['itemid', 'Category']]

In [0]:
submission.to_csv('submission28.csv', index = False)